In [2]:
from dotenv import dotenv_values
import os

env_vars = dotenv_values('.env')

os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY');

In [4]:

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    chat.predict("What is the Korea?")
    print(usage)

Tokens Used: 93
	Prompt Tokens: 12
	Completion Tokens: 81
Successful Requests: 1
Total Cost (USD): $0.00018
